In [1]:
import json, os, tqdm

dataset_json_path = '/data2/chence/PanoHeadData/single_view_hq/dataset_v2.json'
khs_train_paths = {}
khs_val_paths = {}
ours_paths = {}
with open(dataset_json_path, 'r') as f:
    dataset = json.load(f)

not_shown_ours = True
not_shown_khs = True
for image_name, image_meta in tqdm.tqdm(dataset.items()):
    source = image_meta['data_source']
    if source == 'OCD/Original':
        ours_paths[image_name] = image_meta['align_image_path']
    elif source == 'K-Hairstyle/Validation':
        new_path = image_meta['align_image_path'].replace('align_images', 'validation_labels')
        if new_path.endswith('._00.png'): 
            new_path = new_path.replace('._00.png', '.json')
        else:
            new_path = new_path.replace('_00.png', '.json')
        new_path = new_path.replace('-', '_')
        khs_val_paths[image_name] = new_path
    elif source == 'K-Hairstyle/Training':
        new_path = image_meta['align_image_path'].replace('align_images', 'training_labels')
        if new_path.endswith('._00.png'): 
            new_path = new_path.replace('._00.png', '.json')
        else:
            new_path = new_path.replace('_00.png', '.json')
        new_path = new_path.replace('-', '_')
        khs_train_paths[image_name] = new_path


100%|██████████| 264441/264441 [00:00<00:00, 1210635.98it/s]


In [2]:
vis_num = 10
label_root_dir = '/data3/khs_labels/'
attr_name = 'length'

attrs = set()
attr_samples = {}

for k, v in tqdm.tqdm(khs_val_paths.items()):
    abs_path = os.path.join(label_root_dir, v)
    with open(abs_path, 'r') as f:
        label = json.load(f)
        attr = label[attr_name]
        attrs.add(attr)
        if attr not in attr_samples:
            attr_samples[attr] = [k]
        else:
            attr_samples[attr].append(k)
for k, v in tqdm.tqdm(khs_train_paths.items()):
    abs_path = os.path.join(label_root_dir, v)
    with open(abs_path, 'r') as f:
        label = json.load(f)
        attr = label[attr_name]
        attrs.add(attr)
        if attr not in attr_samples:
            attr_samples[attr] = [k]
        else:
            attr_samples[attr].append(k)
print(f'Number of attributes: {len(attr_samples)}')
for attr_cat, attr_val in attr_samples.items():
    print(f'Attribute {attr_cat}: Numbers {len(attr_val)}')

100%|██████████| 29907/29907 [00:10<00:00, 2845.57it/s]

Number of attributes: 5
Attribute 단발: Numbers 10802
Attribute 남자: Numbers 15732
Attribute 중발: Numbers 3002
Attribute 장발: Numbers 1327
Attribute 여숏: Numbers 2368


In [15]:
print(len(dataset))

264441


In [18]:
from PIL import Image
from IPython.display import display
selected_dataset = 'Web'
selected_view = 'back'
back_view_num = 0
vis_samples = 40
image_root_dir = '/data2/chence/PanoHeadData/single_view_hq/align_images'
views = {'back':0, 'front':0, 'left':0, 'right':0}
for image_name, image_meta in tqdm.tqdm(dataset.items()):
    source = image_meta['data_source']
    view = image_meta['view']
    if not(source.startswith(selected_dataset) and view == selected_view):
        dataset[image_name]['dup_num'] = 1
    else:
        dataset[image_name]['dup_num'] = 10
with open(dataset_json_path, 'w') as f:
    json.dump(dataset, f, indent=4)


100%|██████████| 264441/264441 [00:00<00:00, 910239.50it/s]


In [ ]:
# Visualize some basestyles given that exceptional is 해당없음
from IPython.display import display
from matplotlib import pyplot as plt
from PIL import Image
image_root_dir = '/data2/chence/PanoHeadData/single_view_hq/align_images'
num_vis = 20
for k, v in attr_samples.items():
    print(f'Samples of {k}')
    vised = 0
    fig, axs = plt.subplots(5, 4, figsize=(16, 12))
    axs = axs.flatten()
    for _v in attr_samples[k][::50]:
        image_path = os.path.join(image_root_dir, _v.replace('png', 'jpg'))
        image = Image.open(image_path)
        print(f'{vised}: {image_path}')
        axs[vised].imshow(image)
        axs[vised].axis('off')
        vised += 1
        if num_vis == vised: 
            plt.show()
            break

In [ ]:
from IPython.display import display
from matplotlib import pyplot as plt
from PIL import Image
plt.rcParams['font.family'] = 'DejaVu Sans'
image_root_dir = '/data2/chence/PanoHeadData/single_view_hq/align_images'
for k, v in hairstyle_samples.items():
    v = v.replace('png', 'jpg')
    image_path = os.path.join(image_root_dir, v)
    image = Image.open(image_path)
    print(k)
    # specify height and width of figure in pixels
    plt.figure(figsize=(2,2))
    plt.imshow(image)
    plt.show()

In [20]:
for k, v in exceptional_samples.items():
    v = v.replace('png', 'jpg')
    image_path = os.path.join(image_root_dir, v)
    image = Image.open(image_path)
    print(k)
    # specify height and width of figure in pixels
    # plt.figure(figsize=(2,2))
    # plt.imshow(image)
    # plt.show()

해당없음
묶음머리
넘긴머리
악성곱슬머리
땋은머리
기타악세사리
군인머리(반삭)
기타


In [22]:
suspected_samples = {
    '해당없음': [],
    '묶음머리': [],
    '넘긴머리': [],
    '악성곱슬머리': [],
    '땋은머리': [],
    '기타악세사리': [],
    '군인머리(반삭)': [],
    '기타': []
}
for k, v in tqdm.tqdm(khs_val_paths.items()):
    abs_path = os.path.join(label_root_dir, v)
    with open(abs_path, 'r') as f:
        label = json.load(f)
        exceptional = label['exceptional']
        if exceptional in suspected_samples.keys():
            suspected_samples[exceptional].append(k)
for k, v in tqdm.tqdm(khs_train_paths.items()):
    abs_path = os.path.join(label_root_dir, v)
    with open(abs_path, 'r') as f:
        label = json.load(f)
        exceptional = label['exceptional']
        if exceptional in suspected_samples.keys():
            suspected_samples[exceptional].append(k)
for k, v in suspected_samples.items():
    print(k, len(v))

  4%|▍         | 511/11931 [00:00<00:04, 2560.86it/s]

100%|██████████| 109816/109816 [00:46<00:00, 2338.92it/s]

해당없음 107026
묶음머리 6673
넘긴머리 5351
악성곱슬머리 656
땋은머리 960
기타악세사리 758
군인머리(반삭) 227
기타 96


In [ ]:
'''
Author: tianhao 120090472@link.cuhk.edu.cn
Date: 2023-11-12 16:12:08
LastEditors: tianhao 120090472@link.cuhk.edu.cn
LastEditTime: 2023-11-14 13:59:04
FilePath: /DatProc/X10.get_ours_and_khs.ipynb
Description: 

Copyright (c) 2023 by ${git_name_email}, All Rights Reserved. 
'''
for k in suspected_samples.keys():
    if k != '해당없음': continue
    print(f'Samples of {k}')
    num_vis = 10
    for _v in suspected_samples[k][::40]:
        image_path = os.path.join(image_root_dir, _v.replace('png', 'jpg'))
        print(image_path)
        image = Image.open(image_path)
        plt.figure(figsize=(2,2))
        plt.imshow(image)
        plt.show()
        num_vis -= 1
        if num_vis == 0: break


In [ ]:
# read dataset.json
import json, os, tqdm

delete_items = []
label_root_dir = '/data3/khs_labels/'
dataset_json_path = '/data2/chence/PanoHeadData/single_view_hq/dataset.json'
suspected_samples = ['기타악세사리', '땋은머리']

print(f'Loading present dataset meta from: {dataset_json_path}')
with open(dataset_json_path, 'r') as f:
    dataset = json.load(f)

# get datasource
print(f'Filtering KHS data...')
pbar = tqdm.tqdm(dataset.items())
for image_name, image_meta in pbar:
    source = image_meta['data_source']
    if not(source.startswith('K-Hairstyle')): continue
    path_label = 'validation_labels' if source == 'K-Hairstyle/Validation' else 'training_labels'
    new_path = image_meta['align_image_path'].replace('align_images', path_label)
    new_path = new_path.replace('._00.png', '.json') if new_path.endswith('._00.png') else new_path.replace('_00.png', '.json')
    new_path = new_path.replace('-', '_')
    abs_path = os.path.join(label_root_dir, new_path)
    pbar.set_description(f'Processing {new_path}')
    with open(abs_path, 'r') as f:
        label = json.load(f)
        exceptional = label['exceptional']
        if exceptional in suspected_samples:
            delete_items.append(image_name)

print(f'In total: {len(delete_items)} items to be deleted.')


In [1]:
'''
Author: tianhao 120090472@link.cuhk.edu.cn
Date: 2023-11-14 00:15:44
LastEditors: tianhao 120090472@link.cuhk.edu.cn
LastEditTime: 2023-11-14 01:10:58
FilePath: /DatProc/X10.remove_unwanted_khs.py
Description: 

Copyright (c) 2023 by ${git_name_email}, All Rights Reserved. 
'''
# read dataset.json
import json, os, tqdm

ours_delete = []
khs_delete = []
label_root_dir = '/data3/khs_labels/'
dataset_json_path = '/data2/chence/PanoHeadData/multi_view_hq/dataset.json'
suspected_samples = ['기타악세사리', '땋은머리']
samples = [(1224, 7), (3102, 0), (3929, 0), (4033, 0), (4032,3), (4288, 2), (4289, 0), (4473, 0)]

print(f'Loading present dataset meta from: {dataset_json_path}', end='...', flush=True)
with open(dataset_json_path, 'r') as f:
    dataset = json.load(f)
print(f'Done.')



Loading present dataset meta from: /data2/chence/PanoHeadData/multi_view_hq/dataset.json...Done.


In [ ]:
# get datasource
print(f'Filtering KHS data...')
pbar = tqdm.tqdm(samples)
for model_id, image_id in pbar:
    key = f'{model_id:05d}/img{image_id:08d}.png'
    assert key in dataset.keys()
    source = dataset[key]['data_source']
    path_label = 'validation_labels' if source == 'K-Hairstyle/Validation' else 'training_labels'
    new_path = dataset[key]['align_image_path'].replace('align_images', path_label)
    new_path = new_path.replace('._00.png', '.json') if new_path.endswith('._00.png') else new_path.replace('_00.png', '.json')
    new_path = new_path.replace('-', '_')
    abs_path = os.path.join(label_root_dir, new_path)
    pbar.set_description(f'Processing {new_path}')
    with open(abs_path, 'r') as f:
        label = json.load(f)
        style = label['basestyle']
        exceptional = label['exceptional']
        print(f'{key}:\n\tStyle:{style}\n\tExceptional:{exceptional}')
    pbar.update()


In [10]:
headings = 4
model_ids = set()
with open('./temp2.txt','r') as f:
    lines = f.readlines()
    for line in lines[headings:]:
        model_id = os.path.basename(os.path.dirname(line))
        model_ids.add(model_id) 
print(len(model_ids))

184


In [ ]:
# sides = set()
# side_samples= {}
# hairstyles = set()
# exceptionals = set()
# hairstyle_samples = {}
# exceptional_samples = {}
# for k, v in tqdm.tqdm(khs_val_paths.items()):
#     abs_path = os.path.join(label_root_dir, v)
#     with open(abs_path, 'r') as f:
#         label = json.load(f)
#         exceptional = label['exceptional']
#         if not(exceptional == '해당없음'): continue
#         hairstyle = label['basestyle']
#         hairstyles.add(hairstyle)
#         if hairstyle not in hairstyle_samples:
#             hairstyle_samples[hairstyle] = [k]
#         else:
#             hairstyle_samples[hairstyle].append(k)
# for k, v in tqdm.tqdm(khs_train_paths.items()):
#     abs_path = os.path.join(label_root_dir, v)
#     with open(abs_path, 'r') as f:
#         label = json.load(f)
#         exceptional = label['exceptional']
#         if not(exceptional == '해당없음'): continue
#         hairstyle = label['basestyle']
#         hairstyles.add(hairstyle)
#         if hairstyle not in hairstyle_samples:
#             hairstyle_samples[hairstyle] = [k]
#         else:
#             hairstyle_samples[hairstyle].append(k)